# Analyzing borrowers’ risk of defaulting



On the following document we are going to prepare and clean the dataset in order to build a credit score for a potential customer based on the data provided to us. 

## Open the data file and have a look at the general information. 



In [1]:
# Loading all the libraries
import pandas as pd
import warnings
warnings.filterwarnings('ignore')



## Task 1. Data exploration

**Description of the data**
- `children` - the number of children in the family
- `days_employed` - work experience in days
- `dob_years` - client's age in years
- `education` - client's education
- `education_id` - education identifier
- `family_status` - marital status
- `family_status_id` - marital status identifier
- `gender` - gender of the client
- `income_type` - type of employment
- `debt` - was there any debt on loan repayment
- `total_income` - monthly income
- `purpose` - the purpose of obtaining a loan

In [2]:
# Let's see how many rows and columns our dataset has
print(df.shape)


(21525, 12)


There are 21525 rows and that the are 12 columns.

In [3]:
# let's print the first 10 rows
print(df.head(10))


   children  days_employed  dob_years            education  education_id  \
0         1   -8437.673028         42    bachelor's degree             0   
1         1   -4024.803754         36  secondary education             1   
2         0   -5623.422610         33  Secondary Education             1   
3         3   -4124.747207         32  secondary education             1   
4         0  340266.072047         53  secondary education             1   
5         0    -926.185831         27    bachelor's degree             0   
6         0   -2879.202052         43    bachelor's degree             0   
7         0    -152.779569         50  SECONDARY EDUCATION             1   
8         2   -6929.865299         35    BACHELOR'S DEGREE             0   
9         0   -2188.756445         41  secondary education             1   

       family_status  family_status_id gender income_type  debt  total_income  \
0            married                 0      F    employee     0     40620.102   
1

From the print of the first 10 rows we can see the following things:
*There are negative values in days_employed variable, which is odd and does not make sense.
*There are typo's of values in the education variable that we will need to replace.
*Although family_status_id and education_id, debt are integers they represent categorical value.
*No one of the first 10 people in the data has any debt.
*There are lots of different causes for obtaining a loan and some of them are duplicates, we will have to replace duplicates and maybe categorize them. 

In [4]:
# Get info on data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


There are 2 columns with missing values (days_employed, total_income).

In [5]:
# Let's look in the filtered table at the the first column with missing data
print(df.loc[df['days_employed'].isna()])


       children  days_employed  dob_years            education  education_id  \
12            0            NaN         65  secondary education             1   
26            0            NaN         41  secondary education             1   
29            0            NaN         63  secondary education             1   
41            0            NaN         50  secondary education             1   
55            0            NaN         54  secondary education             1   
...         ...            ...        ...                  ...           ...   
21489         2            NaN         47  Secondary Education             1   
21495         1            NaN         50  secondary education             1   
21497         0            NaN         48    BACHELOR'S DEGREE             0   
21502         1            NaN         42  secondary education             1   
21510         2            NaN         28  secondary education             1   

           family_status  family_status

It seems from the print that everytime a value is missing from days_employed variable it is missing at the total_income too, but we cant be sure unless we will filter the data.

In [6]:
# Let's apply multiple conditions for filtering data and look at the number of rows in the filtered table.
print(df.loc[df['days_employed'].isna()& df['total_income'].isna()].count())
print(df.loc[df['days_employed'].notnull()& df['total_income'].isna()])

children            2174
days_employed          0
dob_years           2174
education           2174
education_id        2174
family_status       2174
family_status_id    2174
gender              2174
income_type         2174
debt                2174
total_income           0
purpose             2174
dtype: int64
Empty DataFrame
Columns: [children, days_employed, dob_years, education, education_id, family_status, family_status_id, gender, income_type, debt, total_income, purpose]
Index: []


First, I printed a count of values when the data is filtered to the missing values in both columns. Since we know that there are (21525-19351=) 2174 missing values in each column, and we see here that when we filter, there are 2174 rows left, we can say that the missing values are symmetric. 
To be sure, I printed also a filtered data when only one of the columns has missing values and the other one is not - the df is empty, meaning that there are not rows that include missing total_income value and regular days_employed value.
 

In [7]:
print("{:.2%}".format(len(df.loc[df['days_employed'].isna()& df['total_income'].isna()])/df.shape[0]))

10.10%


Drawing from the last print, we know that clients that have missing values regarding their income, have missing data about their days of employment. Since it is only 10% of the dataset we will try to look at the possible charectaristics - since both variables are connected to employement and income we will check weather there is a connection to the type of income (maybe all the missing values are for entreprenuer?).

In [8]:
# Let's investigate clients who do not have data on identified characteristic and the column with the missing values
print("Missing values by type of income:")
print(df.loc[df['days_employed'].isna()& df['total_income'].isna()]['income_type'].sort_values().value_counts())



Missing values by type of income:
income_type
employee         1105
business          508
retiree           413
civil servant     147
entrepreneur        1
Name: count, dtype: int64


In [9]:
# Checking distribution

print("Checking the ratio of type of income between missing values and the whole data:")
print(df.loc[df['days_employed'].isna()]['income_type'].sort_values().value_counts()*100/df['income_type'].value_counts())

Checking the ratio of type of income between missing values and the whole data:
income_type
business                        9.990167
civil servant                  10.075394
employee                        9.937944
entrepreneur                   50.000000
paternity / maternity leave          NaN
retiree                        10.710581
student                              NaN
unemployed                           NaN
Name: count, dtype: float64


In [10]:
# Checking the distribution in the whole dataset

print("Checking the distribution of type of income on the whole data:")
print(df.loc[df['days_employed'].notnull()]['income_type'].sort_values().value_counts()*100/sum(df['income_type'].value_counts()))

Checking the distribution of type of income on the whole data:
income_type
employee                       46.522648
business                       21.263647
retiree                        15.995354
civil servant                   6.095238
unemployed                      0.009292
entrepreneur                    0.004646
paternity / maternity leave     0.004646
student                         0.004646
Name: count, dtype: float64


We see that missing data constitutes 50% of the entrepreneurs, 9% of business, 10% of civil servants, 9% of employees and 10% of retiree, compared to the whole dataset. Most of the missing data are employees, and then business and retiree; but still there is no particular pattern that may suggest a convenient reason for the missing values.


**Intermediate conclusion**
The distribution does not look the same, it seems like the missing values are random and do not represent something else. 
To make sure that it is not because of the marital status of people, or the number of children:


In [11]:
print("Missing values by number of children:")
print(df.loc[df['days_employed'].isna()]['children'].sort_values().value_counts())

Missing values by number of children:
children
 0     1439
 1      475
 2      204
 3       36
 20       9
 4        7
-1        3
 5        1
Name: count, dtype: int64


In [12]:
print("Missing values by family status:")
print(df.loc[df['days_employed'].isna()]['family_status'].sort_values().value_counts())

Missing values by family status:
family_status
married              1237
civil partnership     442
unmarried             288
divorced              112
widow / widower        95
Name: count, dtype: int64


We see no pattern in the number of children or family status.

**Conclusions**

I think that all the missing values are random. For this reason I plan to change the values in the rows with the missing values to median or mean in the total_income column (because it is a relevant variable to our main hypotheses). In addition, I will find and delete duplicate rows, replace case sensitive duplicate values and check the variables for odd or unreasonble values.

## Data transformation

Checking the values in education column:

In [13]:
# Let's see all values in education column to check if and what spellings will need to be fixed
df['education'].sort_values().unique()

array(["BACHELOR'S DEGREE", "Bachelor's Degree", 'GRADUATE DEGREE',
       'Graduate Degree', 'PRIMARY EDUCATION', 'Primary Education',
       'SECONDARY EDUCATION', 'SOME COLLEGE', 'Secondary Education',
       'Some College', "bachelor's degree", 'graduate degree',
       'primary education', 'secondary education', 'some college'],
      dtype=object)

In [14]:
# Fix the registers
def replace_names (dict):
    for key, value in dict.items():
            df['education']= df['education'].replace(value,key)
education_dict = {"bachelor's degree": ["BACHELOR'S DEGREE", "Bachelor's Degree"],
                "graduate degree": ['GRADUATE DEGREE','Graduate Degree'],
                "primary education": ['PRIMARY EDUCATION', 'Primary Education'],
                'secondary education':['SECONDARY EDUCATION','Secondary Education'],
                 'some college': ['SOME COLLEGE', 'Some College', 'some collage']}
replace_names(education_dict)

In [15]:
# Checking all the values in the column to make sure we fixed them
df['education'].unique()


array(["bachelor's degree", 'secondary education', 'some college',
       'primary education', 'graduate degree'], dtype=object)

Checking the data the `children` column

In [16]:
# Let's see the distribution of values in the `children` column
df['children'].sort_values().value_counts()


children
 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: count, dtype: int64

In [17]:
print("{:.2%}".format(len(df.loc[df['children']==20])/df.shape[0]))

0.35%


In [18]:
print("{:.2%}".format(len(df.loc[df['children']==-1])/df.shape[0]))

0.22%


We can see two problematic things- there are 47 people with negative number of children (which is not possible) and 76 people with 20 kids (which is possible but less likely to happen). Both values account for less than 1% of the whole data and the variable is one of our main questions regarding paing on time, that is why we will delete all these rows. 

In [19]:
df.shape[0]

21525

In [20]:
# [fix the data based on your decision]
filtered_df = df[df['children']<20]
filtered_df = filtered_df[filtered_df['children']>-1]


In [21]:
filtered_df.shape[0]

21402

In [22]:
# Checking the `children` column again to make sure it's all fixed
filtered_df['children'].sort_values().value_counts()


children
0    14149
1     4818
2     2055
3      330
4       41
5        9
Name: count, dtype: int64

Checking the data in the `days_employed` column. 

In [23]:
# Find problematic data in `days_employed`, if they exist, and calculate the percentage
filtered_df['days_employed'].sort_values().value_counts()

days_employed
-18388.949901     1
-569.391678       1
-570.087127       1
-570.661581       1
-570.825590       1
                 ..
-2105.905773      1
-2106.520236      1
-2106.630695      1
-2107.091980      1
 401755.400475    1
Name: count, Length: 19240, dtype: int64

In [24]:
print("{:.2%}".format(len(filtered_df.loc[filtered_df['days_employed']<0])/filtered_df.shape[0]))

73.87%


Since the problematic data constitutes 74% of the data (plus we have missing values) we will fix it now. 

In [25]:
filtered_df['days_employed'].describe() 

count     19240.000000
mean      63159.820777
std      140928.943329
min      -18388.949901
25%       -2747.235601
50%       -1203.934202
75%        -289.740178
max      401755.400475
Name: days_employed, dtype: float64

The variable looks irrational because there is no value of negative days employed (because it means- not employed at all and that is 0 days). Additionally even the minimum values do not make any sense because it is too many days for a lifetime of someone. I believe that since most of the data has negative values and big values it represents a coding mistake and values should be treated as absolute, and they represent hours and not days.

In [26]:
#changing the values in the original df
filtered_df['days_employed']=filtered_df['days_employed'].abs()

In [27]:
filtered_df['days_employed']

0          8437.673028
1          4024.803754
2          5623.422610
3          4124.747207
4        340266.072047
             ...      
21520      4529.316663
21521    343937.404131
21522      2113.346888
21523      3112.481705
21524      1984.507589
Name: days_employed, Length: 21402, dtype: float64

In [28]:
filtered_df['days_employed'].describe()

count     19240.000000
mean      67027.691459
std      139130.846446
min          24.141633
25%         927.984311
50%        2195.251592
75%        5556.372075
max      401755.400475
Name: days_employed, dtype: float64

In [29]:
max_days= 60*365
max_days

21900

In [30]:
filtered_df.loc[filtered_df['days_employed']>max_days, 'days_employed'] = max_days
filtered_df['days_employed'].describe()       

count    19240.000000
mean      5839.283574
std       7768.428022
min         24.141633
25%        927.984311
50%       2195.251592
75%       5556.372075
max      21900.000000
Name: days_employed, dtype: float64

Let's now look at the client's age: 

In [31]:
# Check the `dob_years` for suspicious values and count the percentage

filtered_df['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75], dtype=int64)

In [32]:
print("{:.2%}".format(len(filtered_df.loc[filtered_df['dob_years']==0])/filtered_df.shape[0]))

0.47%


WE have only one value that is odd = 0, which means that the age of the client is zero. Since it is less than 1% we will leave it that way.

Check the `family_status` column:

In [33]:
# Let's see the values for the column
filtered_df['family_status'].value_counts().sort_values()


family_status
widow / widower        952
divorced              1189
unmarried             2799
civil partnership     4160
married              12302
Name: count, dtype: int64

The variable looks fine.


Checking the `gender` column:

In [34]:
# Let's see the values in the column
filtered_df['gender'].value_counts()

gender
F      14154
M       7247
XNA        1
Name: count, dtype: int64

In [35]:
# we see only one value that is not female or male. and its name makes it seems like a typo - XNA instead of NA. Since it is only one value and not very important to our analysis we will change it to NA
filtered_df.loc[filtered_df['gender']=='XNA', 'gender']=  'N/A'

In [36]:
# Check the result - make sure it's fixed
filtered_df['gender'].value_counts()


gender
F      14154
M       7247
N/A        1
Name: count, dtype: int64

Now let's check the `income_type` column:

In [37]:
# Let's see the values in the column
filtered_df['income_type'].value_counts()

income_type
employee                       11050
business                        5054
retiree                         3839
civil servant                   1453
unemployed                         2
entrepreneur                       2
student                            1
paternity / maternity leave        1
Name: count, dtype: int64

The variable looks fine.

Now let's see if we have any duplicates in our data:

In [38]:
# Checking duplicates

filtered_df.duplicated().value_counts()

False    21331
True        71
Name: count, dtype: int64

In [39]:
print(filtered_df[filtered_df.duplicated()])

       children  days_employed  dob_years            education  education_id  \
2849          0            NaN         41  secondary education             1   
3290          0            NaN         58  secondary education             1   
4182          1            NaN         34    bachelor's degree             0   
4851          0            NaN         60  secondary education             1   
5557          0            NaN         58  secondary education             1   
...         ...            ...        ...                  ...           ...   
20702         0            NaN         64  secondary education             1   
21032         0            NaN         60  secondary education             1   
21132         0            NaN         47  secondary education             1   
21281         1            NaN         30    bachelor's degree             0   
21415         0            NaN         54  secondary education             1   

           family_status  family_status

In [40]:
# Address the duplicates
filtered_df2= filtered_df.drop_duplicates(inplace=False)

In [41]:
# Last check whether we have any duplicates
filtered_df2.duplicated().value_counts()

False    21331
Name: count, dtype: int64

In [42]:
# Check the size of the dataset that you now have after your first manipulations with it
filtered_df2.shape[0]

21331

In [43]:
filtered_df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21331 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21331 non-null  int64  
 1   days_employed     19240 non-null  float64
 2   dob_years         21331 non-null  int64  
 3   education         21331 non-null  object 
 4   education_id      21331 non-null  int64  
 5   family_status     21331 non-null  object 
 6   family_status_id  21331 non-null  int64  
 7   gender            21331 non-null  object 
 8   income_type       21331 non-null  object 
 9   debt              21331 non-null  int64  
 10  total_income      19240 non-null  float64
 11  purpose           21331 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


In [44]:
print("{:.2%}".format(filtered_df2.shape[0]/df.shape[0]))

99.10%


Our new dataset - filtered_df2 does not have duplicated rows, is 99.10% of the original data - meaning that we did not lose many data, and has missing values that we adressed earlier in days_employed, and total_income.


# Working with missing values

### Restoring missing values in `total_income`

Next, we will adress the columns with the missing values. First, we will manage the missing values in the total income variable - we will check if there is a different variable that his different categories can explain different values in total income, and then use the median or mean of each category to fill in estimated values. First we'll check age, and to do that we will categorize it:

In [45]:
# Let's write a function that calculates the age category
def assign_age_group(age):
    if age == pd.isna(age):
        return 'NA'
    elif age < 10:
        return '0-9'
    elif 10<= age <20:
        return '10-19'
    elif 20<= age <30:
        return '20-29'
    elif 30<= age <40:
        return '30-39'
    elif 40<= age <50:
        return '40-49'
    elif 50<= age <60:
        return '50-59'
    elif 60<= age <70:
        return '60-69'
    elif 70<= age:
        return '70+'

# Apply function assign_age_group    
filtered_df2['age_group']= filtered_df2['dob_years'].apply(assign_age_group)
    

In [46]:
# Checking how values in the new column
filtered_df2['age_group'].value_counts()


age_group
30-39    5625
40-49    5318
50-59    4631
20-29    3152
60-69    2322
70+       169
NA        100
10-19      14
Name: count, dtype: int64

Next, we will create a table that only has data without missing values. This data will be used to restore the missing values and based on it we will calculate the means/medians.

In [47]:
ommited_df= filtered_df2.dropna().reset_index(drop=True)

In [48]:
ommited_df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,40-49
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,30-39
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,30-39
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,30-39
4,0,21900.000000,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,50-59
5,0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,20-29
6,0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,40-49
7,0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education,50-59
8,2,6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,30-39
9,0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,40-49


Since the job type is a good indicator for income, and sadly there are significant differences between men and women in their average income, I will use these to fill the missing values in total_income variable:

In [49]:
#we will look at the charectaristics of age and type of income as factors for total income:
grouped_income =ommited_df.groupby(['gender','income_type']).agg({'total_income':['mean', 'median']})
grouped_income

total_income            
                                            mean      median
gender income_type                                          
F      business                     29486.338306  25724.8570
       civil servant                24903.993017  21912.6450
       employee                     23811.530355  20883.5075
       entrepreneur                 79866.103000  79866.1030
       paternity / maternity leave   8612.661000   8612.6610
       retiree                      21459.996281  18521.4620
       unemployed                   32435.602000  32435.6020
M      business                     37338.049328  31522.3825
       civil servant                33973.783911  29650.4325
       employee                     28956.174998  25967.9690
       retiree                      24115.492778  20900.1500
       student                      15712.260000  15712.2600
       unemployed                    9593.119000   9593.1190
N/A    business                     32624.825000  32624.8250

There are differences between the median and mean values in some of the categories, so it is better to use the median to fill the missing values.

In [50]:
#Using median value of total income grouped by income type to fill in missing values: 
filtered_df2['total_income']= filtered_df2['total_income'].fillna(filtered_df2.groupby(['gender','income_type'])['total_income'].transform('median'))
#checking for missing values
filtered_df2['total_income'].isna().value_counts()

total_income
False    21330
True         1
Name: count, dtype: int64

In [51]:
filtered_df2[filtered_df2['total_income'].isna()] #checking manually the problematic row

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
5936,0,NaN,58,bachelor's degree,0,married,0,M,entrepreneur,0,NaN,buy residential real estate,50-59


Aparently, this row still has missing value in total income because we didn't know how to fill it - there is no male who is entrepreneur in the ommited data so we do not have a similar case. there are big differences between type of income so for that reason I chose put manually the median of the total income of females that work in the same type of job.

In [52]:
filtered_df2['total_income'][5936]=79866.1030

In [53]:
filtered_df2['total_income'].isna().value_counts()

total_income
False    21331
Name: count, dtype: int64

###  Restoring values in `days_employed`

type of income and age are connected to the duration that people keep in a job, so I will use them to characterize the missing value.

In [54]:
grouped_days =ommited_df.groupby(['age_group','income_type']).agg({'days_employed':['mean', 'median']})
grouped_days

days_employed              
                                               mean        median
age_group income_type                                            
10-19     business                       525.137538    695.968951
          civil servant                  509.969922    509.969922
          employee                       810.376486    796.983636
20-29     business                      1135.505126    916.841875
          civil servant                 1588.676786   1348.765084
          employee                      1204.081971   1011.419361
          entrepreneur                   520.848083    520.848083
          retiree                      21900.000000  21900.000000
          student                        578.751554    578.751554
30-39     business                      1859.178354   1536.208255
          civil servant                 2759.727301   2590.601064
          employee                      2008.764547   1526.991507
          paternity / maternity leave   3296.759962   3296.759962
          retiree                      21900.000000  21900.000000
          unemployed                   21900.000000  21900.000000
40-49     business                      2484.633228   1903.675535
          civil servant                 4040.286376   3566.857698
          employee                      2669.896204   1904.189181
          retiree                      21900.000000  21900.000000
          unemployed                   21900.000000  21900.000000
50-59     business                      2815.097597   2026.398537
          civil servant                 4842.345727   3826.493631
          employee                      3252.677580   2223.483821
          retiree                      21900.000000  21900.000000
60-69     business                      3477.081000   2358.275817
          civil servant                 4433.159823   3339.663548
          employee                      3900.165279   2669.390397
          retiree                      21900.000000  21900.000000
70+       business                      5292.598849   3095.344969
          civil servant                 2811.783573   1678.969771
          employee                      3310.691322   1504.924191
          retiree                      21900.000000  21900.000000
NA        business                      1788.865798   1072.409395
          civil servant                 5878.760924   4945.165547
          employee                      1901.919712   1478.092467
          retiree                      21900.000000  21900.000000

In [55]:
# Since there are difference between the mean and median we'll choose the median because it is safer and less affected by outliers
#Now we'll fill in missing values in the original df by median values of days employed by age group
filtered_df2['days_employed']= filtered_df2['days_employed'].fillna(filtered_df2.groupby(['age_group', 'income_type'])['days_employed'].transform('median'))
#checking for missing values
filtered_df2['days_employed'].isna().value_counts()

days_employed
False    21330
True         1
Name: count, dtype: int64

In [56]:
filtered_df2[filtered_df2['days_employed'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
5936,0,NaN,58,bachelor's degree,0,married,0,M,entrepreneur,0,79866.103,buy residential real estate,50-59


In [57]:
#Again the same man is problematic because there is no another entrepreneur in this age, also there aren't examples for
#people who work as entrepreneurs in 60-69 or 40-49 ages, so we'll calculate a median of the days employed for the age group 
#and use it.
med_days_employed= ommited_df.loc[ommited_df['age_group']== '50-59','days_employed'].median()
med_days_employed
#filling the missing value
filtered_df2['days_employed'][5936]=med_days_employed

In [58]:
filtered_df2['days_employed'].isna().value_counts()

days_employed
False    21331
Name: count, dtype: int64

In [59]:
# Check the entries in all columns - make sure we fixed all missing values
filtered_df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21331 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21331 non-null  int64  
 1   days_employed     21331 non-null  float64
 2   dob_years         21331 non-null  int64  
 3   education         21331 non-null  object 
 4   education_id      21331 non-null  int64  
 5   family_status     21331 non-null  object 
 6   family_status_id  21331 non-null  int64  
 7   gender            21331 non-null  object 
 8   income_type       21331 non-null  object 
 9   debt              21331 non-null  int64  
 10  total_income      21331 non-null  float64
 11  purpose           21331 non-null  object 
 12  age_group         21331 non-null  object 
dtypes: float64(2), int64(5), object(6)
memory usage: 2.8+ MB


## Categorization of data

Our hypotheses are focused on the precence of children (or the amount of them), marital status, income level and type of loan pupose. We know from the work until now that the marital status categorized; we need to categorize the purposes and children then the income level in order to answer our questions.
We will start with loan purposes:


In [60]:
# Print the unique values
filtered_df2['purpose'].unique()


array(['purchase of the house', 'car purchase', 'supplementary education',
       'to have a wedding', 'housing transactions', 'education',
       'having a wedding', 'purchase of the house for my family',
       'buy real estate', 'buy commercial real estate',
       'buy residential real estate', 'construction of own property',
       'property', 'building a property', 'buying a second-hand car',
       'buying my own car', 'transactions with commercial real estate',
       'building a real estate', 'housing',
       'transactions with my real estate', 'cars', 'to become educated',
       'second-hand car purchase', 'getting an education', 'car',
       'wedding ceremony', 'to get a supplementary education',
       'purchase of my own house', 'real estate transactions',
       'getting higher education', 'to own a car', 'purchase of a car',
       'profile education', 'university education',
       'buying property for renting out', 'to buy a car',
       'housing renovation', 'going

It seems like the main topics are: housing, car, education, wedding,

In [61]:
# Check the unique values
def rename_purposes(purpose):
    if 'car' in purpose:
        return 'car'
    elif 'educ' in purpose:
        return 'education'
    elif 'wed' in purpose:
        return 'wedding'
    elif 'hous' or 'estat' or 'propert' in purpose:
        return 'real estate'
    else:
        return 'other'

In [62]:
# Let's write a function to categorize the data based on common topics
filtered_df2['purpose_cat']= filtered_df2['purpose'].apply(rename_purposes)

In [63]:
# checking the categories of purpose:
filtered_df2['purpose_cat'].unique()


array(['real estate', 'car', 'education', 'wedding'], dtype=object)

In [64]:
filtered_df2['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

In [65]:
def assign_children(children):
    if pd.isna(children):
        return 'NA'
    elif children== 0:
        return 'none'
    elif children>0:
        return 'yes'

In [66]:
filtered_df2['parenting']= filtered_df2['children'].apply(assign_children)

In [67]:
filtered_df2.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,purpose_cat,parenting
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,40-49,real estate,yes
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,30-39,car,yes
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,30-39,real estate,none
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,30-39,education,yes
4,0,21900.000000,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,50-59,wedding,none
5,0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,20-29,real estate,none
6,0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,40-49,real estate,none
7,0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education,50-59,education,none
8,2,6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,30-39,wedding,yes
9,0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,40-49,real estate,none


Now we'll try to categorize the income level:

In [68]:
# Getting summary statistics for the column
filtered_df2['total_income'].describe()

count     21331.000000
mean      26456.292631
std       15749.762581
min        3306.762000
25%       17201.214500
50%       23059.392000
75%       31522.382500
max      362496.645000
Name: total_income, dtype: float64

We can see that the variable is skewed and there are few outliers with big numbers, so we will make few categories based on our info about the quartiles


In [69]:
 #Creating function for categorizing into different numerical groups based on ranges
def assign_income_level(income):
    income=income['total_income']
    if income < 0 or pd.isna(income):
        return 'NA'
    elif income < 10000:
        return '0-9.99K'
    elif 10000<= income <20000:
        return '10K-19.99K'
    elif 20000<= income <30000:
        return '20K-29.99K'
    elif 30000<= income <40000:
        return '30K-39.99K'
    elif 40000<= income <50000:
        return '40K-49.99K'
    elif 50000<= income:
        return '50K+'
    


In [70]:
#applying the function on new column
filtered_df2['income_level']= filtered_df2.apply(assign_income_level, axis=1)


In [71]:
# Count each categories values to see the distribution
filtered_df2['income_level'].value_counts().sort_values()

income_level
0-9.99K        921
50K+          1317
40K-49.99K    1480
30K-39.99K    3269
10K-19.99K    6735
20K-29.99K    7609
Name: count, dtype: int64

## Checking the Hypotheses


**Is there a correlation between having children and paying back on time?**

In [72]:
# Check the children data and paying back on time
children_grouped= filtered_df2.groupby('parenting').agg({'debt': ['count', 'sum']})
children_grouped['ratio']=children_grouped['debt']['sum']/children_grouped['debt']['count']
print(children_grouped)


            debt           ratio
           count   sum          
parenting                       
none       14091  1063  0.075438
yes         7240   669  0.092403


In [73]:
# Calculating default-rate based on the number of children
children_grouped2= filtered_df2.groupby('children').agg({'debt': ['count', 'sum']})
children_grouped2['ratio']=children_grouped2['debt']['sum']/children_grouped2['debt']['count']
print(children_grouped2)

           debt           ratio
          count   sum          
children                       
0         14091  1063  0.075438
1          4808   444  0.092346
2          2052   194  0.094542
3           330    27  0.081818
4            41     4  0.097561
5             9     0  0.000000


**Conclusion**

First, we wanted to answer to the question whether there is a connection between having kids and repaying loan on time, so we checked if having kids (vs. not having kids at all) effects the ratio of defaulting a debt. We see that people without kids do not repay debt on time 7% of the time while those with kids do not repay 9% of the time. Meaning that, having kids is lowering the chance of repaying debt on time.
Then, we wanted to check the rate of repaying debt across different number on children. We see that there is no significant difference between having 1 or 2 kids, that having 3 kids is better than having 1,2 or 4  and having 4 children has the highest rates of defaulting lawn, but is based on small numbers. 

**Is there a correlation between family status and paying back on time?**

In [74]:
# Check the family status data and paying back on time
family_grouped= filtered_df2.groupby('family_status').agg({'debt': ['count', 'sum']})
family_grouped['ratio']=family_grouped['debt']['sum']/family_grouped['debt']['count']
print(family_grouped)


                    debt          ratio
                   count  sum          
family_status                          
civil partnership   4134  385  0.093130
divorced            1189   84  0.070648
married            12261  927  0.075606
unmarried           2796  273  0.097639
widow / widower      951   63  0.066246


**Conclusion**

From the results above we can see that widows have the highest chance to pay back in time, and only 6% of them do not pay in time; unmarried people have the highest rate of people that do not pay loan in time, with 9.7% and right after them are people in civil partnership. 

**Is there a correlation between income level and paying back on time?**

In [75]:
# Check the income level data and paying back on time
income_grouped= filtered_df2.groupby('income_level').agg({'debt': ['count', 'sum']})
income_grouped['ratio']=income_grouped['debt']['sum']/income_grouped['debt']['count']
print(income_grouped)


              debt          ratio
             count  sum          
income_level                     
0-9.99K        921   58  0.062975
10K-19.99K    6735  576  0.085523
20K-29.99K    7609  651  0.085557
30K-39.99K    3269  253  0.077394
40K-49.99K    1480  102  0.068919
50K+          1317   92  0.069856


**Conclusion**
We do not see a correlation between income level and paying back, because having larger income level does not mean that the chance of paying debt in time is higher.People in low income level (under 10K) or high income level (40K+) paying more debt on time than those in the middle.

**How does credit purpose affect the default rate?**

In [76]:
# Check the percentages for default rate for each credit purpose and analyze them
purpose_grouped= filtered_df2.groupby('purpose_cat').agg({'debt': ['count', 'sum']})
purpose_grouped['ratio']=purpose_grouped['debt']['sum']/purpose_grouped['debt']['count']
print(purpose_grouped)


              debt          ratio
             count  sum          
purpose_cat                      
car           4279  400  0.093480
education     3492  326  0.093356
real estate  11247  823  0.073175
wedding       2313  183  0.079118


**Conclusion**
From the results above we can say that people that ask for loan for wedding or real-estate are more likely to pay back in time than those that ask for loan to buy a car or paying on education. 


# General Conclusion 

The database included missing values regarding income and time of employment, these missing values were filled with median values of income by type of job and days of employment by age.Additionally there were problematic values that were fixed - people with age of 0, negative and enormous values of days of employment. Later we deleted duplice rows and categorized values for the tests.
From the data we can say that clients without kids are more likely to pay the loan on time compared to clients with kids.

Clients that are unmarried or live in civil partenrship are less likely to pay on time compared to married or divorced and these are less likely to pay in time compared to widows. Meaning that the more 'advanced' family status of the client is, the chance the he will pay back on time is higher.
Clients with low level of income, and high levels of income (40K+) have higher chance of paying on time compared to clients in the middle levels. 
Clients that ask for a loan to buy a car or pay for education are less likely to pay on time, compared to those that ask for a loan for wedding or real-estate.

In conclusion, the clients with the best score should be married/divorced/widowers, without children, that want to wed or buy a house and earn less than 10K or higher than 40K. The clients with the lowest scores shouled be unmarried or in civil partnership, having kids, that want to pay for education or to buy a car, and earn 10-40K.


